In [3]:
import sys, os
%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
#%matplotlib notebook
%matplotlib tk

import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
from foundation import util
import torch.multiprocessing as mp
import h5py as hf
import time

/home/fleeb/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/fleeb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/fleeb/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/fleeb/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
arm = fd.envs.Arm()

root = 'data/'
dirs = ['test_0.h5']

In [5]:
data = hf.File(os.path.join(root, dirs[0]))

In [6]:
for k in data.keys():
    print(k, data[k].shape)

controls (4, 5, 2)
labels (4, 6)
lengths (2,)
masses (2,)
rgbs (4, 6)
states (4, 6, 4)


In [7]:
plt.figure()
plt.imshow(util.str_to_byte_img(data['labels'][1,0]))

In [20]:
rgbs = [util.str_to_rgb(data['rgbs'][1,i]) for i in range(6)]
rgb.shape

(128, 128, 3)

In [21]:
fig, axes = plt.subplots(6)
for rgb, ax in zip(rgbs, axes):
    ax.imshow(rgb)

In [17]:
data['controls'][:]

array([[[-0.38515437,  0.26815736],
        [-0.01981318,  0.7928895 ],
        [-0.08874404,  0.26461256],
        [-0.30221307, -0.19656539],
        [-0.9553485 , -0.6622821 ]],

       [[-0.05677128, -0.8797674 ],
        [ 0.5437317 ,  0.48739874],
        [ 0.18884242,  0.7757133 ],
        [-0.09793735,  0.59898126],
        [-0.7003182 , -0.19706237]],

       [[ 0.42718768,  0.23807275],
        [-0.11493468, -0.80846274],
        [ 0.2283144 , -0.8853446 ],
        [ 0.131428  ,  0.06645787],
        [-0.21989834,  0.8176969 ]],

       [[-0.9414369 ,  0.59971726],
        [-0.20572531,  0.5087439 ],
        [ 0.13901699, -0.12244415],
        [ 0.27736092,  0.04933178],
        [ 0.36522818, -0.389701  ]],

       [[-0.6779411 , -0.43546283],
        [ 0.36321712,  0.83038795],
        [-0.20580018,  0.74831176],
        [-0.16118336,  0.1058141 ],
        [ 0.9054762 , -0.92767036]],

       [[ 0.6946373 ,  0.74971247],
        [ 0.29662085, -0.5704845 ],
        [ 0.89861 

In [3]:
ds = []
for d in dirs:
    d = os.path.join(root, d)
    if not os.path.exists(d):
        print('{} does not exist'.format(d))
    
    fs = [f for f in os.listdir(d) if '.pth.tar' in f][:1]
    
    print('found {} data files in {}'.format(len(fs), d))
    
    ds.extend([torch.load(os.path.join(d, f)) for f in fs])

excluded_keys = {'masses', 'lengths'}
raw_data = {}
for k in ds[0].keys():
    if k not in excluded_keys:
        raw_data[k] = torch.cat([d[k] for d in ds])
    else:
        raw_data[k] = ds[0][k]
    print('{} : {}'.format(k, raw_data[k].size()))
del ds

found 1 data files in data/strong_arm_seq50
found 1 data files in data/strong_arm_seq50_1
found 1 data files in data/strong_arm_seq50_2
found 1 data files in data/strong_arm_seq50_3
actions : torch.Size([4000, 50, 2])
states : torch.Size([4000, 51, 8])
lengths : torch.Size([2])
masses : torch.Size([2])


In [4]:
seq_len = 10
tdata = TensorDataset(raw_data['states'][:,:seq_len+1], raw_data['actions'][:,:seq_len])

In [14]:
def arm_render(x, y, gx, gy, ht, wd, arm_args={}, base_args={}, hand_args={}, goal_args={}):
    plt.switch_backend('Agg') # offscreen rendering
    
    fig, ax = plt.subplots(figsize=(ht/100, wd/100))
    ax.set_xlim(-1.05,1.05)
    ax.set_ylim(-1.05,1.05)

    ax.plot([gx], [gy], **goal_args)

    ax.plot(x, y, **arm_args)
    ax.plot([0], [0], **base_args)
    ax.plot(x[-1:], y[-1:], **hand_args)
    ax.add_artist(plt.Circle((0, 0), 1, color='r', lw=2, fill=False)) # boundary

    fig.subplots_adjust(bottom=0., left=0., right=1., top=1.) # full sized image
    rgb = util.fig_to_rgba(fig)[:,:,:-1]

    plt.close(fig)
    del fig
    del ax
    
    lbl_color = lambda n: '#{}'.format("{0:#0{1}x}".format(n,8)[2:])
    n = 0

    colors = [(0,0,0), (1,0,0), (0,1,0), (0,0,1)]
    lbl_color = lambda n: colors[n]

    fig, ax = plt.subplots(figsize=(ht/100, wd/100))
    ax.set_xlim(-1.05, 1.05)
    ax.set_ylim(-1.05, 1.05)

    bg = lbl_color(n)  # 0
    n += 1
    ax.add_artist(plt.Circle((0, 0), 2, color=bg, fill=True))  # boundary

    goal_lbl_args = goal_args.copy()
    goal_lbl_args['color'] = lbl_color(n)
    n+=1

    ax.plot([gx], [gy], **goal_lbl_args)

    link_colors = [lbl_color(i) for i in range(n,n+2)]
    n += 2

    arm_lbl_args = arm_args.copy()
    mk = arm_lbl_args['marker']
    arm_lbl_args['marker'] = ''

    for i, (ex, ey, sx, sy, c) in enumerate(zip(x,y,x[1:], y[1:], link_colors)):
        arm_lbl_args['color'] = c
        ax.plot([sx,ex], [sy,ey], **arm_lbl_args)
        if i == 0:
            base_lbl_args = base_args.copy()
            base_lbl_args['color'] = bg
            ax.plot([0], [0], **base_lbl_args)

    arm_lbl_args['ls'] = ''
    arm_lbl_args['marker'] = mk

    for sx, sy, c in zip(x[1:], y[1:], link_colors):
        arm_lbl_args['color'] = c
        ax.plot([sx], [sy], **arm_lbl_args)

    hand_lbl_args = hand_args.copy()
    hand_lbl_args['color'] = arm_lbl_args['color'] # color of last
    ax.plot(x[-1:], y[-1:], **hand_lbl_args)

    fig.subplots_adjust(bottom=0., left=0., right=1., top=1.)  # full sized image
    lbl = util.fig_to_rgba(fig)[:,:,:-1]
    bg = lbl.sum(2) == 0
    lbl = lbl.argmax(2) + 1
    lbl[bg] = 0

    plt.close(fig)
    del fig
    del ax
    
    return rgb, lbl

def format_arm(states, actions, step_len=1, seq_len=None, n_links=2, lengths=None, moving_target=True, 
               imgs=True, img_ht=200, img_wd=200, xy_grid=None, render_args={}): # (seq-len, (n-links+2)*2)
    
    states = states[::step_len]
    actions = actions[::step_len]
    
    if seq_len is not None:
        states = states[:seq_len+1]
        actions = actions[:seq_len]
    else:
        seq_len = actions.size(0)
        
    if lengths is None:
        lengths = torch.ones(n_links)
        lengths /= lengths.sum()
    lengths = lengths.unsqueeze(0)
    
    sample = {
        'states': states,
        'actions': actions,    
    }
    
    x, y = torch.cos(states[:, :n_links]).mul(lengths).cumsum(-1), torch.sin(states[:, :n_links]).mul(lengths).cumsum(-1)
    x, y = torch.cat([torch.zeros(seq_len+1,1), x], -1), torch.cat([torch.zeros(seq_len+1,1), y], -1)
    
    gx, gy = states[:,n_links] * torch.cos(states[:,n_links+1]), \
            states[:,n_links] * torch.sin(states[:,n_links+1])
    
    poses = torch.zeros(seq_len+1, n_links+1, 3) # [goal, link1, link2]
    poses[:, 0, 0], poses[:, 0, 1] = gx, gy
    poses[:, 1:, 0], poses[:, 1:, 1] = x[:, :-1], y[:, :-1]
    poses[:, 1:, 2] = states[:, :n_links]
    poses[:,:,2] %= 2 * np.pi
    sample['gtposes'] = poses
    
    if not imgs:
        return sample
    
    rgbs, labels = [], []
    for pos in zip(x.numpy(), y.numpy(), gx.numpy(), gy.numpy()):
        rgb, label = arm_render(*pos, ht=img_ht, wd=img_wd, **render_args)
        rgbs.append(torch.from_numpy(rgb))
        labels.append(torch.from_numpy(label))
    
    sample['rgbs'] = torch.stack(rgbs).permute(0,3,1,2)
    sample['labels'] = torch.stack(labels)
    
    num_masks = sample['labels'][0].max() + 1
    sample['gtmasks'] = torch.zeros(seq_len+1, num_masks, img_ht, img_wd)
    
    for m in range(num_masks):
        sample['gtmasks'][:,m] = sample['labels'] == m
    
    
    
    return sample

In [15]:
img_wd, img_ht = 128, 128
data = fd.data.Format_Dataset(tdata, format_arm, format_args={
    'seq_len': 10,
    'img_wd': img_wd,
    'img_ht': img_ht,
    'render_args': {
        'base_args':{'color':'k', 'marker':'o', 'ms':7},
        'arm_args':{'color':'k', 'ls':'-', 'lw':2.5, 'marker':'o', 'ms':4},
        'goal_args':{'color':'#2ca02c', 'marker':'o', 'ms':6},
        'hand_args':{'color': '#1f77b4', 'marker':'o', 'ms':6}
    }
})
data.xy_grid = torch.stack([torch.linspace(-1,1,img_wd).view(1, -1).expand(img_ht, img_wd),
                           torch.linspace(-1,1,img_ht).view(-1, 1).expand(img_ht, img_wd)]).unsqueeze(0)

In [16]:
data_loader = DataLoader(data, batch_size=3, shuffle=False, num_workers=4)
samples = iter(data_loader)

In [17]:
sample = next(samples)

for k in sample.keys():
    print('{:<12} : {}'.format(k, sample[k].size()))

states       : torch.Size([3, 11, 8])
actions      : torch.Size([3, 10, 2])
gtposes      : torch.Size([3, 11, 3, 3])
rgbs         : torch.Size([3, 11, 3, 128, 128])
labels       : torch.Size([3, 11, 128, 128])
gtmasks      : torch.Size([3, 11, 4, 128, 128])


In [18]:
sample['gtposes'][0,0]

tensor([[ 0.3449,  0.4849,  0.0000],
        [ 0.0000,  0.0000,  2.6169],
        [-0.4327,  0.2505,  3.3186]])

In [19]:
plt.switch_backend('TkAgg')
plt.ion()
plt.figure()
plt.imshow(sample['rgbs'][0,0].permute(1,2,0).numpy())
plt.figure()
plt.imshow(sample['labels'][0,0].numpy())
fig, axes = plt.subplots(2,2)
axes = axes.flat
for m, ax in zip(sample['gtmasks'][0,0].numpy(), axes):
    ax.imshow(m)

In [ ]:
plt.close('all')